In [1]:
import pandas as pd
import numpy as np
import tqdm
import warnings
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score

pd.set_option("max_rows", 200)
pd.set_option("max_columns", 50)
tqdm.tqdm.pandas()
%matplotlib inline

In [2]:
df = pd.read_pickle("../input/riiid-test-answer-prediction/split10/train_0.pickle").sort_values(["user_id", "timestamp"]).reset_index(drop=True)

In [3]:
import sys
sys.path.append("../")
# df = pd.read_pickle("../input/riiid-test-answer-prediction/split10/train_0.pickle")
df = df.sort_values(["user_id", "timestamp"])
# large_user_id = df["user_id"].value_counts()
# large_user_id = large_user_id[large_user_id > 1000] 
# df = df[df["user_id"].isin(large_user_id.index)]
# df = df[df["answered_correctly"] != -1]
df["answered_correctly"] = df["answered_correctly"].replace(-1, np.nan)

In [22]:
df[df["content_type_id"] == 1]["content_id"].value_counts()

3153     3759
21411    2743
10540    2469
29695    2409
15888    2318
         ... 
20852       1
23239       1
25876       1
9831        1
22353       1
Name: content_id, Length: 408, dtype: int64

In [36]:
content_id = 3153

In [37]:
df["leced"] = (df["content_type_id"] == 1).astype(int) * (df["content_id"] == content_id).astype(int)
df["lec_count"] = df.groupby("user_id")["leced"].cumsum().replace(0, np.nan)
df["final_lec_idx"] = df.groupby(["user_id", "lec_count"]).cumcount()
df[(df["final_lec_idx"] < 100)].groupby("final_lec_idx")["answered_correctly"].describe().head(100)

,count,mean,std,min,25%,50%,75%,max
final_lec_idx,,,,,,,,
0,1.0,1.000000,NaN,1.0,1.0,1.0,1.0,1.0
1,3738.0,0.788122,0.408694,0.0,1.0,1.0,1.0,1.0
2,3739.0,0.803156,0.397667,0.0,1.0,1.0,1.0,1.0
3,3401.0,0.650103,0.477007,0.0,0.0,1.0,1.0,1.0
4,3545.0,0.658392,0.474316,0.0,0.0,1.0,1.0,1.0
5,3503.0,0.657436,0.474635,0.0,0.0,1.0,1.0,1.0
6,3433.0,0.649286,0.477263,0.0,0.0,1.0,1.0,1.0
7,3466.0,0.661858,0.473145,0.0,0.0,1.0,1.0,1.0
8,3449.0,0.661931,0.473121,0.0,0.0,1.0,1.0,1.0


In [38]:
df[df["final_lec_idx"] == 1].groupby("content_id")["answered_correctly"].describe().sort_values("count", ascending=False)

,count,mean,std,min,25%,50%,75%,max
content_id,,,,,,,,
1315,383.0,0.738903,0.439807,0.0,0.0,1.0,1.0,1.0
738,203.0,0.733990,0.442962,0.0,0.0,1.0,1.0,1.0
762,126.0,0.769841,0.422615,0.0,1.0,1.0,1.0,1.0
660,100.0,0.770000,0.422953,0.0,1.0,1.0,1.0,1.0
553,93.0,0.720430,0.451220,0.0,0.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...
13605,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16136,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21852,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [45]:
df[(df["content_id"]==1315) & (df["content_type_id"] == 0)]["answered_correctly"].describe()

count    1491.000000
mean        0.363514
std         0.481172
min         0.000000
25%         0.000000
50%         0.000000
75%         1.000000
max         1.000000
Name: answered_correctly, dtype: float64

In [42]:
df_oof2 = pd.read_csv("../output/ex_172/20201202080625/oof_train_0_lgbm.csv")
df_oof2.columns = ["index", "predict_lgbm", "target"]

In [43]:
df = pd.merge(df.reset_index(), df_oof2, how="inner")

In [50]:
df[(df["content_id"]==1315) & (df["content_type_id"] == 0)].groupby("final_lec_idx")[["answered_correctly", "predict_lgbm"]].describe()

answered_correctly                                               \
                           count      mean       std  min  25%  50%  75%  max   
final_lec_idx                                                                   
1                           65.0  0.723077  0.450961  0.0  0.0  1.0  1.0  1.0   
2                           57.0  0.736842  0.444262  0.0  0.0  1.0  1.0  1.0   
3                            9.0  0.555556  0.527046  0.0  0.0  1.0  1.0  1.0   
4                            7.0  0.714286  0.487950  0.0  0.5  1.0  1.0  1.0   
5                            5.0  0.600000  0.547723  0.0  0.0  1.0  1.0  1.0   
...                          ...       ...       ...  ...  ...  ...  ...  ...   
7191006                      1.0  0.000000       NaN  0.0  0.0  0.0  0.0  0.0   
7208456                      1.0  0.000000       NaN  0.0  0.0  0.0  0.0  0.0   
7220314                      1.0  1.000000       NaN  1.0  1.0  1.0  1.0  1.0   
7220363                      1.0  0.000000       NaN  0.0  0.0  0.0  0.0  0.0   
7221866                      1.0  0.000000       NaN  0.0  0.0  0.0  0.0  0.0   

              predict_lgbm                                                    \
                     count      mean       std       min       25%       50%   
final_lec_idx                                                                  
1                     65.0  0.697514  0.283010  0.036089  0.471298  0.818484   
2                     57.0  0.701207  0.210616  0.141728  0.551801  0.777922   
3                      9.0  0.645382  0.261671  0.237024  0.437641  0.720486   
4                      7.0  0.404365  0.234063  0.160581  0.182622  0.381130   
5                      5.0  0.290870  0.061402  0.194250  0.271863  0.315403   
...                    ...       ...       ...       ...       ...       ...   
7191006                1.0  0.258929       NaN  0.258929  0.258929  0.258929   
7208456                1.0  0.291405       NaN  0.291405  0.291405  0.291405   
7220314                1.0  0.647121       NaN  0.647121  0.647121  0.647121   
7220363                1.0  0.204958       NaN  0.204958  0.204958  0.204958   
7221866                1.0  0.271743       NaN  0.271743  0.271743  0.271743   

                                   
                    75%       max  
final_lec_idx                      
1              0.928873  0.972142  
2              0.848259  0.972561  
3              0.830071  0.956813  
4              0.625937  0.671725  
5              0.318623  0.354210  
...                 ...       ...  
7191006        0.258929  0.258929  
7208456        0.291405  0.291405  
7220314        0.647121  0.647121  
7220363        0.204958  0.204958  
7221866        0.271743  0.271743  

[1300 rows x 16 columns]